In [ ]:
import sympy as sp
sp.init_printing()
import IPython.display as disp
%matplotlib ipympl
import matplotlib.pyplot as plt
#plt.rcParams['text.usetex'] = True
import numpy as np

# 2D
# Continuous piecewise bilinear functions
drawing = f"""
        _  .___._____.
        |  |   |     |
        |  |   |     |
    hy+ |  |   |     |
        |  |   |     |
        _  .___._____.
        |  |   |     |
    hy- |  |   |     |
        _  .___._____.
        
           |___|_____|
             hx-  hx+
             
"""
print(drawing)

In [ ]:
# !!! Indicates a hypothesis (should be studied and considered)

# !!! Mode selection
mode_values = "real" # real or complex values

Wavenumber variables

In [ ]:
# Physical wavenumbers
k = sp.Symbol('k',
              #real=True, # !!!
              #positive=True, # !!!
              )

Coordinate variables

In [ ]:
from sympy.vector import CoordSys3D
sys = CoordSys3D('sys', vector_names=['i', 'j', 'k'])
x, y, z = sys.x, sys.y, sys.z

Galerkine variational forms

In [ ]:
from sympy.vector import gradient


def a_G(u, v, xa, xb, ya, yb):
    
    # gradients
    grad_u = gradient(u)
    grad_v = gradient(v)
    
    return sp.integrate(grad_u & grad_v, (x, xa, xb), (y, ya, yb)) - (k**2 * sp.integrate(u * v, (x, xa, xb), (y, ya, yb)))


Grid sizes

In [ ]:
h_x_, h_y_ = dict(), dict()
h_x_["-"], h_x_["+"], h_y_["-"], h_y_["+"] = sp.symbols('h_x^- h_x^+ h_y^- h_y^+')

Grid points centered around origin

In [ ]:
x_, y_ = dict(), dict()
x_["-1"], x_["0"], x_["+1"] = -h_x_["-"], sp.sympify(0), +h_x_["+"]
y_["-1"], y_["0"], y_["+1"] = -h_y_["-"], sp.sympify(0), +h_y_["+"]

Point-wise bilinear shape functions

In [ ]:
N_x_, N_y_ = dict(), dict()

N_x_["-1"] = - x / h_x_["-"]
N_x_["0-"] = (x / h_x_["-"]) + 1
N_x_["0+"] = (- x / h_x_["+"]) + 1
N_x_["+1"] = x / h_x_["+"]

N_y_["-1"] = - y / h_y_["-"]
N_y_["0-"] = (y / h_y_["-"]) + 1
N_y_["0+"] = (- y / h_y_["+"]) + 1
N_y_["+1"] = y / h_y_["+"]

N_ = dict()

In [ ]:
HX_M, HX_P, HY_M, HY_P = 0.8, 1.1, 0.7, 1

In [ ]:
# Coordinate pairs generation via cartesian products, over each quadrant

num = 10

X_M = np.linspace(-HX_M, 0, num=num)
X_P = np.linspace(0, +HX_P, num=num)
Y_M = np.linspace(-HY_M, 0, num=num)
Y_P = np.linspace(0, +HY_P, num=num)

XY_MM = np.transpose([np.tile(X_M, len(Y_M)), np.repeat(Y_M, len(X_M))])
XY_MP = np.transpose([np.tile(X_M, len(Y_P)), np.repeat(Y_P, len(X_M))])
XY_PM = np.transpose([np.tile(X_P, len(Y_M)), np.repeat(Y_M, len(X_P))])
XY_PP = np.transpose([np.tile(X_P, len(Y_P)), np.repeat(Y_P, len(X_P))])

In [ ]:
# Bottom-left quadrant shape functions

N_["-1, -1"] = N_x_["-1"] * N_y_["-1"]
N_["-1, 0-"] =  N_x_["-1"] * N_y_["0-"]
N_["0-, -1"] = N_x_["0-"] * N_y_["-1"]
N_["0-, 0-"] =  N_x_["0-"] * N_y_["0-"]

XY = XY_MM

Z0 = []
Z0.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["-"]], expr=N_["-1, -1"])(XY[:,0], XY[:,1], HX_M, HY_M))
Z0.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["-"]], expr=N_["-1, 0-"])(XY[:,0], XY[:,1], HX_M, HY_M))
Z0.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["-"]], expr=N_["0-, -1"])(XY[:,0], XY[:,1], HX_M, HY_M))
Z0.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["-"]], expr=N_["0-, 0-"])(XY[:,0], XY[:,1], HX_M, HY_M))

fig0 = plt.figure(figsize=(3, 3))
ax = fig0.add_subplot(projection='3d')
for Z in Z0:
    ax.scatter(XY[:,0], XY[:,1], Z)
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
# Upper-left quadrant shape functions

N_["-1, 0+"] = N_x_["-1"] * N_y_["0+"]
N_["-1, +1"] =  N_x_["-1"] * N_y_["+1"]
N_["0-, 0+"] = N_x_["0-"] * N_y_["0+"]
N_["0-, +1"] =  N_x_["0-"] * N_y_["+1"]

XY = XY_MP

Z1 = []
Z1.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["+"]], expr=N_["-1, 0+"])(XY[:,0], XY[:,1], HX_M, HY_P))
Z1.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["+"]], expr=N_["-1, +1"])(XY[:,0], XY[:,1], HX_M, HY_P))
Z1.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["+"]], expr=N_["0-, 0+"])(XY[:,0], XY[:,1], HX_M, HY_P))
Z1.append(sp.lambdify(args=[x, y, h_x_["-"], h_y_["+"]], expr=N_["0-, +1"])(XY[:,0], XY[:,1], HX_M, HY_P))

fig0 = plt.figure(figsize=(3, 3))
ax = fig0.add_subplot(projection='3d')
for Z in Z1:
    ax.scatter(XY[:,0], XY[:,1], Z)
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
# Bottom-right quadrant shape functions

N_["0+, -1"] = N_x_["0+"] * N_y_["-1"]
N_["0+, 0-"] =  N_x_["0+"] * N_y_["0-"]
N_["+1, -1"] = N_x_["+1"] * N_y_["-1"]
N_["+1, 0-"] =  N_x_["+1"] * N_y_["0-"]

XY = XY_PM

Z2 = []
Z2.append(sp.lambdify(args=[x, y, h_x_["+"], h_y_["-"]], expr=N_["0+, -1"])(XY[:,0], XY[:,1], HX_P, HY_M))
Z2.append(sp.lambdify(args=[x, y, h_x_["+"], h_y_["-"]], expr=N_["0+, 0-"])(XY[:,0], XY[:,1], HX_P, HY_M))
Z2.append(sp.lambdify(args=[x, y, h_x_["+"], h_y_["-"]], expr=N_["+1, -1"])(XY[:,0], XY[:,1], HX_P, HY_M))
Z2.append(sp.lambdify(args=[x, y, h_x_["+"], h_y_["-"]], expr=N_["+1, 0-"])(XY[:,0], XY[:,1], HX_P, HY_M))

fig0 = plt.figure(figsize=(3, 3))
ax = fig0.add_subplot(projection='3d')
for Z in Z2:
    ax.scatter(XY[:,0], XY[:,1], Z)
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:

# Upper-right quadrant shape functions

N_["0+, 0+"] = N_x_["0+"] * N_y_["0+"]
N_["0+, +1"] =  N_x_["0+"] * N_y_["+1"]
N_["+1, 0+"] = N_x_["+1"] * N_y_["0+"]
N_["+1, +1"] =  N_x_["+1"] * N_y_["+1"]

XY = XY_PP

Z3 = np.zeros((4, len(XY)))
Z3[0] = sp.lambdify(args=[x, y, h_x_["+"], h_y_["+"]], expr=N_["0+, 0+"])(XY[:,0], XY[:,1], HX_P, HY_P)
Z3[1] = sp.lambdify(args=[x, y, h_x_["+"], h_y_["+"]], expr=N_["0+, +1"])(XY[:,0], XY[:,1], HX_P, HY_P)
Z3[2] = sp.lambdify(args=[x, y, h_x_["+"], h_y_["+"]], expr=N_["+1, 0+"])(XY[:,0], XY[:,1], HX_P, HY_P)
Z3[3] = sp.lambdify(args=[x, y, h_x_["+"], h_y_["+"]], expr=N_["+1, +1"])(XY[:,0], XY[:,1], HX_P, HY_P)

print(Z3.sum(axis=0))

fig0 = plt.figure(figsize=(3, 3))
ax = fig0.add_subplot(projection='3d')
for Z in Z3:
    ax.scatter(XY[:,0], XY[:,1], Z)
ax.set_xlabel('x')
ax.set_ylabel('y')


In [ ]:
# ALtogether

fig0 = plt.figure(figsize=(6, 6))
ax = fig0.add_subplot(projection='3d')

XY = XY_MM
for Z in Z0:
    ax.scatter(XY[:,0], XY[:,1], Z)
XY = XY_MP
for Z in Z1:
    ax.scatter(XY[:,0], XY[:,1], Z)
XY = XY_PM
for Z in Z2:
    ax.scatter(XY[:,0], XY[:,1], Z)
XY = XY_PP
for Z in Z3:
    ax.scatter(XY[:,0], XY[:,1], Z)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(left=-HX_M, right=+HX_P)
ax.set_ylim(bottom=-HY_M, top=+HY_P)
ax.set_zlim(bottom=0, top=1)

Linear system coefficients

In [ ]:
A_G00_ = dict()

A_G00_["-1, -1"] = a_G(N_["-1, -1"], N_["0-, 0-"], x_["-1"], x_["0"], y_["-1"], y_["0"]) # bottom-left element shared
A_G00_["-1, 0"] = a_G(N_["-1, 0-"], N_["0-, 0-"], x_["-1"], x_["0"], y_["-1"], y_["0"]) + a_G(N_["-1, 0+"], N_["0-, 0+"], x_["-1"], x_["0"], y_["0"], y_["+1"]) # bottom and upper left elements shared
A_G00_["-1, +1"] = a_G(N_["-1, +1"], N_["0-, 0+"], x_["-1"], x_["0"], y_["0"], y_["+1"]) # upper-left element shared

A_G00_["0, -1"] = a_G(N_["0-, -1"], N_["0-, 0-"], x_["-1"], x_["0"], y_["-1"], y_["0"]) + a_G(N_["0+, -1"], N_["0+, 0-"], x_["0"], x_["+1"], y_["-1"], y_["0"]) # left and right bottom elements shared
A_G00_["0, 0"] = a_G(N_["0-, 0-"], N_["0-, 0-"], x_["-1"], x_["0"], y_["-1"], y_["0"]) + a_G(N_["0-, 0+"], N_["0-, 0+"], x_["-1"], x_["0"], y_["0"], y_["+1"]) + a_G(N_["0+, 0-"], N_["0+, 0-"], x_["0"], x_["+1"], y_["-1"], y_["0"]) + a_G(N_["0+, 0+"], N_["0+, 0+"], x_["0"], x_["+1"], y_["0"], y_["+1"]) # all 4 elements shared
A_G00_["0, +1"] = a_G(N_["0-, +1"], N_["0-, 0+"], x_["-1"], x_["0"], y_["0"], y_["+1"]) + a_G(N_["0+, +1"], N_["0+, 0+"], x_["0"], x_["+1"], y_["0"], y_["+1"]) # left and right upper elements shared

A_G00_["+1, -1"] = a_G(N_["+1, -1"], N_["0+, 0-"], x_["0"], x_["+1"], y_["-1"], y_["0"]) # bottom-right element shared
A_G00_["+1, 0"] = a_G(N_["+1, 0-"], N_["0+, 0-"], x_["0"], x_["+1"], y_["-1"], y_["0"]) + a_G(N_["+1, 0+"], N_["0+, 0+"], x_["0"], x_["+1"], y_["0"], y_["+1"]) # bottom and upper right elements shared
A_G00_["+1, +1"] = a_G(N_["+1, +1"], N_["0+, 0+"], x_["0"], x_["+1"], y_["0"], y_["+1"]) # top-right element shared


In [ ]:
for key, value in A_G00_.items():
    disp.display(sp.Symbol(f'A_{{G00}}^{{{key}}}'))
    # The system is still the same after *(-1)
    disp.display(-value.simplify())
    sp.print_latex(-value)
    print("-" * 20)

Numerical solution form

In [ ]:
k_hx, k_hy = sp.symbols('k_x^h k_y^h')

if mode_values == "complex":
    uh = sp.exp(1j * (k_hx * x + k_hy * y))
elif mode_values == "real":
    uh = sp.cos(k_hx * x + k_hy * y)
    
Uh_ = dict()

Uh_["-1, -1"] = uh.subs([(x, x_["-1"]),
                        (y, y_["-1"])])
Uh_["-1, 0"] = uh.subs([(x, x_["-1"]),
                        (y, y_["0"])])
Uh_["-1, +1"] = uh.subs([(x, x_["-1"]),
                        (y, y_["+1"])])

Uh_["0, -1"] = uh.subs([(x, x_["0"]),
                        (y, y_["-1"])])
Uh_["0, 0"] = uh.subs([(x, x_["0"]),
                        (y, y_["0"])])
Uh_["0, +1"] = uh.subs([(x, x_["0"]),
                        (y, y_["+1"])])

Uh_["+1, -1"] = uh.subs([(x, x_["+1"]),
                        (y, y_["-1"])])
Uh_["+1, 0"] = uh.subs([(x, x_["+1"]),
                        (y, y_["0"])])
Uh_["+1, +1"] = uh.subs([(x, x_["+1"]),
                        (y, y_["+1"])])

Uh_ = {key: sp.expand_trig(Uh_[key]) for key in Uh_.keys()}

Galerkine stencil

In [ ]:
dispersion_relation_G = sum([A_G00_[key]*Uh_[key] for key in A_G00_.keys()])
disp.display(dispersion_relation_G)

Solve the stencil for $k^2$

In [ ]:
k2_G = sp.solve(dispersion_relation_G, k**2)[0]
disp.display(k**2, '=')
disp.display(k2_G)

Replace cosines with letters

In [ ]:
f_x_, f_y_ = dict(), dict()
f_x_["-"], f_x_["+"] = sp.symbols('f_x^- f_x^+')
f_y_["-"], f_y_["+"] = sp.symbols('f_y^- f_y^+')

g_ = dict()
g_["x+, y+"] = sp.symbols('g^{++}')
g_["x+, y-"] = sp.symbols('g^{+-}')
g_["x-, y+"] = sp.symbols('g^{-+}')
g_["x-, y-"] = sp.symbols('g^{--}')



k2_G_fg = k2_G.subs([(sp.cos(k_hx*h_x_["-"]), f_x_["-"]),
                    (sp.cos(k_hx*h_x_["+"]), f_x_["+"]),
                    (sp.cos(k_hy*h_y_["-"]), f_y_["-"]),
                    (sp.cos(k_hy*h_y_["+"]), f_y_["+"]),
                    (sp.cos(h_x_["+"]*k_hx + h_y_["+"]*k_hy), g_["x+, y+"]),
                    (sp.cos(h_x_["+"]*k_hx - h_y_["-"]*k_hy), g_["x+, y-"]),
                    (sp.cos(h_x_["-"]*k_hx - h_y_["+"]*k_hy), g_["x-, y+"]),
                    (sp.cos(h_x_["-"]*k_hx + h_y_["-"]*k_hy), g_["x-, y-"]),])

disp.display(k2_G_fg)
sp.print_latex(k2_G_fg)

Check for consistency with :
- 1D (or 2D with 0° propagation angle) square case

In [ ]:
# Uniform case
h_x, h_y = sp.symbols('h_x h_y')
def uniform(expression):
    return expression.subs([(h_x_["+"], h_x),
                            (h_x_["-"], h_x),
                            (h_y_["+"], h_y),
                            (h_y_["-"], h_y)])

# Aligned wave (along the x axis) case
k_h = sp.symbols('k_h')
def align(expression):
    return expression.subs([(k_hx, k_h),
                            (k_hy, 0)])

# Square case
h = sp.symbols('h')
def square(expression):
    return expression.subs([(h_x_["+"], h),
                            (h_x_["-"], h),
                            (h_y_["+"], h),
                            (h_y_["-"], h)])

print("1D (or 2D with 0° propagation angle) uniform case")
disp.display((k*h)**2, "=")
disp.display(sp.cancel(square(align(k2_G))*h**2))

A simple way of finding $\tau$.

Considering our way of integrating and our elements,
$$
- k^2_{Galerkine} = k^2 (\tau k^2 - 1)
$$

One can determine $\tau k^2$ by replacing the leftmost $k^2$.

In [ ]:
tau = sp.symbols('tau')

tauk2_GLS = sp.solve(k2_G
                      +
                      (k**2)
                      *
                      (tau*(k**2) -1),
                       tau*(k**2))[0]
disp.display(tauk2_GLS)

tauk2_GLS_fg = sp.solve(k2_G_fg
                      +
                      (k**2)
                      *
                      (tau*(k**2) -1),
                       tau*(k**2))[0]
disp.display(tauk2_GLS_fg)

Check for consistency with :
- 1D (or 2D with 0° propagation angle) square case
- 2D with 45° propagation angle square case

In [ ]:
print("1D (or 2D with 0° propagation angle) square case")
disp.display(sp.cancel(square(align(tauk2_GLS))))

print("2D uniform case")
disp.display(sp.expand_trig(uniform(tauk2_GLS)))